In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import random
import os
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

%matplotlib inline

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [3]:
from sklearn import metrics 
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

## Box-Cox 변환

In [86]:

num_cols = ["X_01","X_02","X_03","X_05","X_06","X_07","X_08","X_09","X_10","X_11","X_12","X_13","X_14","X_15","X_16","X_17","X_18","X_19","X_20","X_21","X_22","X_24","X_25","X_26","X_27","X_28","X_29","X_30","X_31","X_32","X_33","X_34","X_55","X_36","X_37","X_38","X_39","X_40","X_41","X_42","X_43","X_44","X_45","X_49","X_50","X_51","X_52","X_53","X_54","X_55","X_56"]
# -----------------------------------
# Box-Cox 변환
# -----------------------------------
# 데이터 읽어오기
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv("./test.csv")

train_df, valid_df = train_test_split(train_df, train_size=0.8,random_state=42)

train_x = train_df.filter(regex='X')
train_y = train_df.filter(regex='Y')
test_x = test_df.filter(regex="X")

valid_x = valid_df.filter(regex='X') # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature
# -----------------------------------

# 양의 정숫값만을 취하는 변수를 변환 대상으로 목록에 저장
# 또한, 결측값을 포함하는 경우는 (~(train_x[c] <= 0.0)).all() 등으로 해야 하므로 주의
pos_cols = [c for c in num_cols if (train_x[c] > 0.0).all() and (test_x[c] > 0.0).all()]
from sklearn.preprocessing import PowerTransformer

# 학습 데이터를 기반으로 복수 열의 박스-칵스 변환 정의
pt = PowerTransformer(method='box-cox')
pt.fit(train_x[pos_cols])

# 변환 후의 데이터로 각 열을 치환
train_x[pos_cols] = pt.transform(train_x[pos_cols])
test_x[pos_cols] = pt.transform(test_x[pos_cols])

# -----------------------------------
# 학습후 리더보드 확인

import xgboost as xgb
XGB = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma = 0, subsample=0.75, colsample_bytree = 1, max_depth=7)).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)

c:\Users\user\anaconda3\envs\(lg_aimers)\lib\site-packages\scipy\stats\morestats.py:908: RuntimeWarning: divide by zero encountered in log
  return (lmb - 1) * np.sum(logdata, axis=0) - N/2 * np.log(variance)
c:\Users\user\anaconda3\envs\(lg_aimers)\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [87]:
print(score)

3.063091732285869


## 여 존슨 변환

In [78]:
num_cols = ["X_01","X_02","X_03","X_05","X_06","X_07","X_08","X_09","X_10","X_11","X_12","X_13","X_14","X_15","X_16","X_17","X_18","X_19","X_20","X_21","X_22","X_24","X_25","X_26","X_27","X_28","X_29","X_30","X_31","X_32","X_33","X_34","X_55","X_36","X_37","X_38","X_39","X_40","X_41","X_42","X_43","X_44","X_45","X_49","X_50","X_51","X_52","X_53","X_54","X_55","X_56"]
# -----------------------------------
# 데이터 읽어오기
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv("./test.csv")

train_df, valid_df = train_test_split(train_df, train_size=0.8,random_state=42)

train_x = train_df.filter(regex='X')
train_y = train_df.filter(regex='Y')
test_x = test_df.filter(regex="X")

valid_x = valid_df.filter(regex='X') # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature
# -----------------------------------

# 양의 정숫값만을 취하는 변수를 변환 대상으로 목록에 저장
# 또한, 결측값을 포함하는 경우는 (~(train_x[c] <= 0.0)).all() 등으로 해야 하므로 주의
pos_cols = [c for c in num_cols if (train_x[c] > 0.0).all() and (test_x[c] > 0.0).all()]
from sklearn.preprocessing import PowerTransformer


# 학습 데이터를 기반으로 복수 열의 여-존슨 변환 정의
pt = PowerTransformer(method='yeo-johnson')
pt.fit(train_x[num_cols])

# 변환 후의 데이터로 각 열을 치환
train_x[num_cols] = pt.transform(train_x[num_cols])
test_x[num_cols] = pt.transform(test_x[num_cols])

# -----------------------------------
# 학습후 리더보드 확인

import xgboost as xgb
XGB = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma = 0, subsample=0.75, colsample_bytree = 1, max_depth=7)).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)


c:\Users\user\anaconda3\envs\(lg_aimers)\lib\site-packages\sklearn\preprocessing\_data.py:2995: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
c:\Users\user\anaconda3\envs\(lg_aimers)\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\user\anaconda3\envs\(lg_aimers)\lib\site-packages\numpy\core\_methods.py:216: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\user\AppData\Roaming\Python\Python38\s

In [79]:
print(score)

3.9661386850591493


# 클리핑 기법

In [5]:
num_cols = ["X_01","X_02","X_03","X_05","X_06","X_07","X_08","X_09","X_10","X_11","X_12","X_13","X_14","X_15","X_16","X_17","X_18","X_19","X_20","X_21","X_22","X_24","X_25","X_26","X_27","X_28","X_29","X_30","X_31","X_32","X_33","X_34","X_55","X_36","X_37","X_38","X_39","X_40","X_41","X_42","X_43","X_44","X_45","X_49","X_50","X_51","X_52","X_53","X_54","X_55","X_56"]
# -----------------------------------
# clipping
# -----------------------------------
# 데이터 읽어오기
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv("./test.csv")

train_df, valid_df = train_test_split(train_df, train_size=0.8,random_state=42)

train_x = train_df.filter(regex='X')
train_y = train_df.filter(regex='Y')
test_x = test_df.filter(regex="X")

valid_x = valid_df.filter(regex='X') # Input : X Featrue
valid_y = valid_df.filter(regex='Y') # Output : Y Feature
# -----------------------------------
# 열마다 학습 데이터의 1%, 99% 지점을 확인
p01 = train_x[num_cols].quantile(0.01)
p99 = train_x[num_cols].quantile(0.99)

p01_valid = valid_x[num_cols].quantile(0.01)
p99_vaild = valid_x[num_cols].quantile(0.99)

# 1％점 이하의 값은 1%점으로, 99%점 이상의 값은 99%점으로 클리핑
train_x = train_x[num_cols].clip(p01, p99, axis=1)
test_x = test_x[num_cols].clip(p01, p99, axis=1)
valid_x = valid_x[num_cols].clip(p01_valid, p99_vaild, axis=1)
# -----------------------------------
# 학습후 리더보드 확인

import xgboost as xgb
XGB = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma = 0, subsample=0.75, colsample_bytree = 1, max_depth=7)).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

1.943799503130345


In [122]:
print(score)

1.943799503130345


## "learning_rate": [0.01],

In [9]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.01],
                    "n_estimators":[ 1000],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=3, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   48.7s remaining:  1.2min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   58.6s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   50.9s remaining:  1.3min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.0min finished


1.9311440378684694


## "learning_rate": [0.009],

In [12]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.009],
                    "n_estimators":[ 1000],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=3, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.2min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.4min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.0min remaining:  1.6min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   48.4s remaining:  1.2min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   57.9s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   46.7s remaining:  1.2min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   56.2s finished


1.9307223843752612


## "learning_rate": [0.008],

In [13]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.008],
                    "n_estimators":[ 1000],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=3, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.6min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.6min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   47.5s remaining:  1.2min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   57.6s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   47.6s remaining:  1.2min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   57.9s finished


1.9314661657620273


In [14]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.007],
                    "n_estimators":[ 1000],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=3, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.6min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   43.2s remaining:  1.1min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   52.2s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   43.3s remaining:  1.1min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:   52.1s finished


1.9321364313145208


# "n_estimators":[ 1100],

In [15]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.008],
                    "n_estimators":[ 1200],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=3, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.2min remaining:  1.8min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.5min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.2min remaining:  1.8min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.5min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   53.2s remaining:  1.3min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.1min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:   53.1s remaining:  1.3min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.1min finished


1.9311421450354906


#             "n_estimators":[ 1200],

In [16]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.008],
                    "n_estimators":[ 1200],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.6min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.6min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.2min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.2min finished


1.930719765257661


#                    "n_estimators":[ 1300],

In [17]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.008],
                    "n_estimators":[ 1300],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

1.9305471840939359


#                     "n_estimators":[ 1400],

In [18]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.008],
                    "n_estimators":[ 1400],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

1.930348430908924


In [20]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.008],
                    "n_estimators":[ 1400],
                    "max_depth": [8],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

1.9303693765278869


In [27]:
import xgboost as xgb
fit_param = {'verbose': False, 'early_stopping_rounds':10, 'eval_set':(train_x, train_y)}
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.001],
                    "n_estimators":[ 1300],
                    "max_depth": [7],
                    "gamma":[ 0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y, fit_param=fit_param)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

TypeError: fit() got an unexpected keyword argument 'fit_param'

In [7]:
import xgboost as xgb
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(seed=42,
                         tree_method='gpu_hist',
                         gpu_id=0),
        param_grid={
                    "learning_rate": [0.08],
                    "n_estimators":[ 1000],
                    "max_depth": [7],
                    "gamma":[ 0.0],
                    "colsample_bytree":[1],
                    "subsample":[0.75],
                    },
                    cv=5, scoring='neg_mean_squared_error', verbose=3, n_jobs=4)

XGB = MultiOutputRegressor(gsc).fit(train_x, train_y)
valid_preds = XGB.predict(valid_x)
score = lg_nrmse(valid_y.values,valid_preds)
print(score)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.4min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  1.3min finished


1.993632614298577


# 제출

In [10]:
preds = XGB.predict(test_x)
submit = pd.read_csv('./sample_submission.csv')
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = preds[:,idx-1]
print('Done.')
submit.to_csv('./submit_XGBoost_clliping.csv', index=False)

Done.
